## Draw Map

In [13]:
import pandas as pd
import folium

# Load the data
file_path = "/Users/reibs/Downloads/Total 72, 05-27-2023, 05 50 PM.csv"  # replace with your file path
data = pd.read_csv(file_path)

# Choose the initial latitude and longitude to center the map, for example, the first point in your data
initial_lat = data.iloc[0]['lat']
initial_long = data.iloc[0]['long']

# Create a map centered at the initial latitude and longitude
m = folium.Map(location=[initial_lat, initial_long], zoom_start=13)

# Prepare locations for polyline
locations = data[['lat', 'long']].values.tolist()

# Add lines between the points
folium.PolyLine(locations, color="red", weight=2.5, opacity=1).add_to(m)

# Save it to a file
m.save('map.html')

## Filter Timestamps

In [34]:
# Required libraries
from geopy.distance import geodesic
import pandas as pd

# Load the csv file
df = pd.read_csv("/Users/reibs/Downloads/RubbishDevelopment/Exports/Total 230, 05-27-2023, 08 20 PM.csv")

# Display the first few rows of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   creationTimeStamp  230 non-null    float64
 1   lat                230 non-null    float64
 2   long               230 non-null    float64
 3   debugIndex         230 non-null    int64  
dtypes: float64(3), int64(1)
memory usage: 7.3 KB


In [35]:
# Convert Unix timestamp to datetime
df['creationTimeStamp'] = pd.to_datetime(df['creationTimeStamp'], unit='s')

# Function to calculate distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).meters

# Initialize a list to hold the filtered data
filtered_data = []

# Add the first row to the filtered data
filtered_data.append(df.iloc[0])

# Iterate through the dataframe, starting from the second row
for i in range(1, len(df)):
    # Get the current and previous row
    current_row = df.iloc[i]
    previous_row = df.iloc[i-1]
    
    # Calculate the distance from the previous row
    distance = calculate_distance(current_row['lat'], current_row['long'], previous_row['lat'], previous_row['long'])
    
    # If the distance is greater than or equal to 1 meter, add the current row to the filtered data
    if distance >= 1:
        filtered_data.append(current_row)

# Convert the filtered data to a dataframe
filtered_df = pd.DataFrame(filtered_data)

filtered_df


,creationTimeStamp,lat,long,debugIndex
0,2023-05-28 03:20:47.557904128,37.795629,-122.394143,229
5,2023-05-28 03:20:41.940603136,37.795615,-122.394130,224
6,2023-05-28 03:20:40.940634112,37.795603,-122.394119,223
7,2023-05-28 03:20:39.940342784,37.795586,-122.394103,222
8,2023-05-28 03:20:38.940450816,37.795561,-122.394080,221
...,...,...,...,...
220,2023-05-28 03:17:06.940716032,37.795712,-122.394193,9
221,2023-05-28 03:17:05.940801024,37.795696,-122.394179,8
222,2023-05-28 03:17:04.940356864,37.795683,-122.394166,7
223,2023-05-28 03:17:03.940630016,37.795674,-122.394155,6


In [37]:
import cv2
import pandas as pd
from pathlib import Path

# Load the CSV file
data = filtered_df
data["creationTimeStamp"] = pd.to_datetime(data["creationTimeStamp"])

# Video file path
video_path = "/Users/reibs/Downloads/RubbishDevelopment/05272023_201652_rubbishvideo.mp4"

# Open the video file
video = cv2.VideoCapture(video_path)

# Get the total number of frames in the video
total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Get the frame rate of the video
frame_rate = video.get(cv2.CAP_PROP_FPS)

# Find the minimum timestamp in the CSV data
min_timestamp = data["creationTimeStamp"].min()

# Create a directory to store the frames
frame_directory = Path("./frames")
frame_directory.mkdir(parents=True, exist_ok=True)

# Initialize a list to store the paths to the saved frames
frame_paths = []

# Iterate over each timestamp in the CSV data in reverse order
for i, timestamp in enumerate(data["creationTimeStamp"][::-1]):
    print(timestamp)
    
    # Subtract the minimum timestamp from the current timestamp and convert to frame number
    frame_number = int((timestamp - min_timestamp).total_seconds() * frame_rate)
    
    # Ensure frame_number does not exceed total frames in the video
    if frame_number >= total_frames:
        break
    
    # Set the current position of the video file to the frame_number
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

    # Read the frame from the video file
    success, frame = video.read()

    # If the frame was read successfully, save it as an image
    if success:
        # Crop the top half of the frame
        height, width = frame.shape[:2]
        cropped_frame = frame[int(height/2):, :]
        
        # Define the path to save the frame
        frame_path = frame_directory / f"frame_{i}.jpg"
        
        # Save the cropped frame as a jpg image
        cv2.imwrite(str(frame_path), cropped_frame)
        
        # Store the path to the saved frame
        frame_paths.append(str(frame_path))

# Release the video file
video.release()

# Print the paths to the saved frames
for path in frame_paths:
    print(path)


2023-05-28 03:17:02.940707072
2023-05-28 03:17:03.940630016
2023-05-28 03:17:04.940356864
2023-05-28 03:17:05.940801024
2023-05-28 03:17:06.940716032
2023-05-28 03:17:07.940689152
2023-05-28 03:17:08.940284928
2023-05-28 03:17:09.940737024
2023-05-28 03:17:10.940638976
2023-05-28 03:17:11.939724800
2023-05-28 03:17:12.940624896
2023-05-28 03:17:13.940805888
2023-05-28 03:17:14.939771904
2023-05-28 03:17:15.939693824
2023-05-28 03:17:16.939697152
2023-05-28 03:17:17.939678976
2023-05-28 03:17:18.940664064
2023-05-28 03:17:19.939650048
2023-05-28 03:17:20.940634112
2023-05-28 03:17:21.940183040
2023-05-28 03:17:22.940115968
2023-05-28 03:17:23.940683008
2023-05-28 03:17:24.939830016
2023-05-28 03:17:25.940572928
2023-05-28 03:17:26.940578816
2023-05-28 03:17:27.940748032
2023-05-28 03:17:28.940675072
2023-05-28 03:17:29.940648192
2023-05-28 03:17:30.940731904
2023-05-28 03:17:31.940429824
2023-05-28 03:17:32.940619008
2023-05-28 03:17:33.940800
2023-05-28 03:17:34.940047872
2023-05-28 03

## Reorg directories

In [38]:
import os
import shutil
from datetime import datetime

# Define the parent directory and the CSV subdirectory
parent_directory = '/Users/reibs/Downloads/RubbishDevelopment/'
csv_subdirectory = '/Users/reibs/Downloads/RubbishDevelopment/Exports/'

# Get list of all mp4 files in the parent directory
mp4_files = [f for f in os.listdir(parent_directory) if f.endswith('.mp4')]

# Get list of all csv files in the CSV subdirectory
csv_files = [f for f in os.listdir(csv_subdirectory) if f.endswith('.csv')]

# Create a dictionary to hold the file lists for each date
date_dict = {}

# Add mp4 files to the dictionary
for file in mp4_files:
    file_path = os.path.join(parent_directory, file)
    mtime = os.path.getmtime(file_path)
    date = datetime.fromtimestamp(mtime).strftime('%Y%m%d')
    
    if date not in date_dict:
        date_dict[date] = {'mp4': [], 'csv': []}
        
    date_dict[date]['mp4'].append(file_path)
    
# Add csv files to the dictionary
for file in csv_files:
    file_path = os.path.join(csv_subdirectory, file)
    mtime = os.path.getmtime(file_path)
    date = datetime.fromtimestamp(mtime).strftime('%Y%m%d')
    
    if date not in date_dict:
        date_dict[date] = {'mp4': [], 'csv': []}
        
    date_dict[date]['csv'].append(file_path)
    
# Move files to their respective directories
for date, files in date_dict.items():
    new_dir = os.path.join(parent_directory, date)
    os.makedirs(new_dir, exist_ok=True)
    
    for file in files['mp4']:
        shutil.move(file, new_dir)
        
    for file in files['csv']:
        shutil.move(file, new_dir)

## Get bounding boxes for frames

In [39]:
import base64
import requests

# Read image file
with open('./frames/frame_0.jpg', 'rb') as f:
    image_data = f.read()

# Encode image in base64
encoded_image = base64.b64encode(image_data)

# Define API endpoint
url = 'https://detect.roboflow.com/litterbug/3?api_key=C04lRGYKvo4X8bpG5cSx'

# Send POST request
response = requests.post(url, data=encoded_image)

# Print response
print(response.json())

{'message': 'Request must include a Content-Type header'}


In [57]:
import json

In [54]:
for i in range(0,10):
    x= !cat ./frames/frame_23.jpg | base64 | curl -d @- "https://detect.roboflow.com/litterbug/3?api_key=C04lRGYKvo4X8bpG5cSx"


In [58]:
json.loads(x[-1])

{'time': 0.05446078199997828,
 'image': {'width': 1080, 'height': 960},
 'predictions': [{'x': 828.5,
   'y': 326.0,
   'width': 57.0,
   'height': 40.0,
   'confidence': 0.5772755742073059,
   'class': 'street-trash'},
  {'x': 879.5,
   'y': 362.0,
   'width': 53.0,
   'height': 30.0,
   'confidence': 0.5566504597663879,
   'class': 'street-trash'},
  {'x': 473.0,
   'y': 197.0,
   'width': 46.0,
   'height': 24.0,
   'confidence': 0.5416112542152405,
   'class': 'street-trash'},
  {'x': 744.0,
   'y': 263.5,
   'width': 38.0,
   'height': 21.0,
   'confidence': 0.5360170602798462,
   'class': 'street-trash'},
  {'x': 1052.0,
   'y': 558.0,
   'width': 48.0,
   'height': 40.0,
   'confidence': 0.4434272050857544,
   'class': 'street-trash'}]}

In [60]:
# The Video
# 05272023_201652_rubbishvideo.mp4
# the CSV
# Total 230, 05-27-2023, 08 20 PM 2.csv

In [59]:
for i in range(0, 10):
    frame_number = i  # Update the frame_number according to your requirement
    output = get_ipython().system(f'cat ./frames/frame_{frame_number}.jpg | base64 | curl -d @- "https://detect.roboflow.com/litterbug/3?api_key=C04lRGYKvo4X8bpG5cSx"')
    results = json.loads(output)

{"time":0.06019305000017994,"image":{"width":1080,"height":960},"predictions":[{"x":633.0,"y":461.0,"width":64.0,"height":54.0,"confidence":0.7062060832977295,"class":"street-trash"},{"x":665.0,"y":551.5,"width":64.0,"height":59.0,"confidence":0.5172853469848633,"class":"street-trash"},{"x":534.0,"y":233.0,"width":24.0,"height":24.0,"confidence":0.47642871737480164,"class":"street-trash"}]}None
{"time":0.06566074500005925,"image":{"width":1080,"height":960},"predictions":[{"x":576.5,"y":272.0,"width":27.0,"height":28.0,"confidence":0.6792889833450317,"class":"street-trash"},{"x":705.0,"y":598.5,"width":70.0,"height":63.0,"confidence":0.6431417465209961,"class":"street-trash"},{"x":673.0,"y":505.0,"width":58.0,"height":50.0,"confidence":0.6207471489906311,"class":"street-trash"},{"x":601.0,"y":331.5,"width":36.0,"height":39.0,"confidence":0.6128804683685303,"class":"street-trash"},{"x":529.0,"y":170.0,"width":24.0,"height":26.0,"confidence":0.5464332699775696,"class":"street-trash"},{"x